In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import sys
import QuantLib as ql

In [2]:
# https://implementingquantlib.substack.com/p/different-kinds-of-swaps

In [3]:
today = ql.Date(19, ql.December, 2024)
ql.Settings.instance().evaluationDate = today

bps = 1e-4

In [4]:
def nparray2listQlDates(today,dates):
    qldates = []
    for i in range(len(dates)):
        if dates[i]>=0:
            qldates.append(today+ql.Period(dates[i], ql.Years))
        else:
            print(- dates[i])
            qldates.append(today-ql.Period(-dates[i], ql.Years))
    return qldates

In [5]:
sofr_curve = ql.ZeroCurve(
    [today, today + ql.Period(50, ql.Years)],
    [0.02, 0.04],
    ql.Actual365Fixed(),
)

In [6]:
#### test
ibordates= [0,1,2,3,4,5,6,7,8,9,10,15,20,30]
ibordatesql2 = nparray2listQlDates(today,ibordates)
iborvaleursnp= np.array([4.,4.418842,4.588199,4.383796,4.442341,4.472094,4.487725,
                       4.603022,4.49304931,4.55046352,4.56296427,4.817077,4.969355, 4.76199173])/100
iborvaleurs=iborvaleursnp.tolist()
sofr_curve = ql.ZeroCurve(
    ibordatesql2,
    iborvaleurs,
    ql.Actual365Fixed()
)
sofr_curve

<QuantLib.QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x752f495334e0> >

In [7]:
sofr_handle = ql.YieldTermStructureHandle(sofr_curve)
sofr = ql.Sofr(sofr_handle)

In [8]:
sofr.fixing(ql.Date(7, ql.February, 2025))

0.04062467653228374

In [9]:
start_date = today
end_date = start_date + ql.Period(10, ql.Years)
coupon_tenor = ql.Period(1, ql.Years)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
convention = ql.Following
rule = ql.DateGeneration.Forward
end_of_month = False

fixed_rate = 40 * bps
fixed_day_counter = sofr.dayCounter()

schedule = ql.Schedule(
    start_date,
    end_date,
    coupon_tenor,
    calendar,
    convention,
    convention,
    rule,
    end_of_month,
)
swap = ql.OvernightIndexedSwap(
    ql.Swap.Payer,
    1_000_000,
    schedule,
    fixed_rate,
    fixed_day_counter,
    sofr,
)

In [10]:
swap.setPricingEngine(
    ql.DiscountingSwapEngine(sofr_handle)
)

In [11]:
swap.NPV()

334614.6680790261

In [12]:
swap.fixedLegNPV()

-31917.749325440538

In [13]:
swap.overnightLegNPV()

366532.41740446666

In [14]:
data = []
for cf in swap.fixedLeg():
    coupon = ql.as_fixed_rate_coupon(cf)
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )

In [15]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"December 19th, 2025",0.40%,1.013889,4055.56
1,"December 21st, 2026",0.40%,1.019444,4077.78
2,"December 20th, 2027",0.40%,1.011111,4044.44
3,"December 19th, 2028",0.40%,1.013889,4055.56
4,"December 19th, 2029",0.40%,1.013889,4055.56
5,"December 19th, 2030",0.40%,1.013889,4055.56
6,"December 19th, 2031",0.40%,1.013889,4055.56
7,"December 20th, 2032",0.40%,1.019444,4077.78
8,"December 19th, 2033",0.40%,1.011111,4044.44
9,"December 19th, 2034",0.40%,1.013889,4055.56


In [16]:
data = []
for cf in swap.overnightLeg():
    coupon = ql.as_floating_rate_coupon(cf)
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )

In [17]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"December 19th, 2025",4.46%,1.013889,45179.27
1,"December 21st, 2026",4.80%,1.019444,48965.57
2,"December 20th, 2027",4.00%,1.011111,40442.25
3,"December 19th, 2028",4.66%,1.013889,47259.30
4,"December 19th, 2029",4.63%,1.013889,46982.14
5,"December 19th, 2030",4.61%,1.013889,46717.66
6,"December 19th, 2031",5.36%,1.013889,54378.19
7,"December 20th, 2032",3.75%,1.019444,38199.73
8,"December 19th, 2033",5.07%,1.011111,51234.54
9,"December 19th, 2034",4.72%,1.013889,47865.66


In [18]:
swap.fairRate()

0.04593461947046684

In [19]:
#   value after issuing

In [20]:
start_date = today - ql.Period(3, ql.Months)
end_date = start_date + ql.Period(10, ql.Years)

schedule = ql.Schedule(
    start_date,
    end_date,
    coupon_tenor,
    calendar,
    convention,
    convention,
    rule,
    end_of_month,
)
swap = ql.OvernightIndexedSwap(
    ql.Swap.Payer,
    1_000_000,
    schedule,
    fixed_rate,
    fixed_day_counter,
    sofr,
)
swap.setPricingEngine(
    ql.DiscountingSwapEngine(sofr_handle)
)

In [21]:
d = ql.Date(21, ql.June, 2023)
while d <= today:
    if sofr.isValidFixingDate(d):
        sofr.addFixing(d, 0.02)
    d += 1

In [22]:
swap.NPV()

331817.7554626634

In [23]:
swap.fixedLegNPV()

-32285.974512246947

In [24]:
swap.overnightLegNPV()

364103.7299749104

In [25]:
data = []
for cf in swap.fixedLeg():
    coupon = ql.as_fixed_rate_coupon(cf)
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )

In [26]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"September 19th, 2025",0.40%,1.013889,4055.56
1,"September 21st, 2026",0.40%,1.019444,4077.78
2,"September 20th, 2027",0.40%,1.011111,4044.44
3,"September 19th, 2028",0.40%,1.013889,4055.56
4,"September 19th, 2029",0.40%,1.013889,4055.56
5,"September 19th, 2030",0.40%,1.013889,4055.56
6,"September 19th, 2031",0.40%,1.013889,4055.56
7,"September 20th, 2032",0.40%,1.019444,4077.78
8,"September 19th, 2033",0.40%,1.011111,4044.44
9,"September 19th, 2034",0.40%,1.013889,4055.56


In [27]:
data = []
for cf in swap.overnightLeg():
    coupon = ql.as_floating_rate_coupon(cf)
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )

In [28]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"September 19th, 2025",3.76%,1.013889,38096.65
1,"September 21st, 2026",4.77%,1.019444,48607.84
2,"September 20th, 2027",4.27%,1.011111,43146.02
3,"September 19th, 2028",4.45%,1.013889,45088.21
4,"September 19th, 2029",4.65%,1.013889,47107.34
5,"September 19th, 2030",4.62%,1.013889,46811.26
6,"September 19th, 2031",5.16%,1.013889,52266.77
7,"September 20th, 2032",4.19%,1.019444,42674.72
8,"September 19th, 2033",4.71%,1.011111,47580.38
9,"September 19th, 2034",4.82%,1.013889,48828.20
